In [0]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
!pip install -q torchbearer
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import numpy as np

# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform) 
testset = MNIST(".", train=False, download=True, transform=transform) 

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)


# define baseline model
class SLP(nn.Module):
    def __init__(self, input_size, num_classes):
      super(SLP, self).__init__()
      self.fc1 = nn.Linear(input_size, num_classes)
    def forward(self, x):
      out = self.fc1(x)
      return out

# build the model
model = SLP(784, 10)

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adagrad(model.parameters())

# Construct a trial object with the model, optimiser and loss.
# Also specify metrics we wish to compute.
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'])

# Provide the data to the trial
trial.with_generators(trainloader, test_generator=testloader)

# Run 5 epochs of training
trial.run(epochs=5)

# test the performance
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)
